# Lab | Comparing regression models


For this lab, we will be using the dataset in the Customer Analysis Business Case of the previous lab. This dataset can be found in files_for_lab folder. In this lab we will explore categorical data.

# Special instructions
As in this lab we will keep working of the same dataset of the previous lab, please make a copy of the final Jupyter notebook of the previous lab in the current lab folder. Next, use Markdown to add a new section in the Jupyter notebook named Lab Comparing Regression Models. Then restart the Kernel and run all the previous cells. Finally, keep working of the same notebook according to the next instructions.

# Instructions

#### General EDA

In [1]:
'''import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, PowerTransformer, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score'''

'import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import OneHotEncoder, PowerTransformer, MinMaxScaler\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.neighbors import KNeighborsRegressor\nfrom sklearn.neural_network import MLPRegressor\nfrom sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score'

In [2]:
# Import neccessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
# Read the data
data = pd.read_csv('files_for_lab/customer_value_analysis.csv')
data.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [4]:
# Rename the columns so they follow the PE8 (snake case)
cols = [i.lower() for i in data.columns]
data.columns = cols 
cols = [i.replace(' ','_') for i in data.columns]
data.columns = cols 
data.head()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


1. Define X and y

In [5]:
# Define X and y
X = data.drop('total_claim_amount', axis=1)
y = data['total_claim_amount']

2. Import sklearn train_test_split and separate the data. Set test_size=0.30 and random_state=31

In [6]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31)

3. Separate X_train and X_test into numerical and categorical (X_train_cat , X_train_num , X_test_cat , X_test_num)

In [7]:
# Separate numerical and categorical features
X_train_cat = X_train.select_dtypes(include=[object])
X_train_num = X_train.select_dtypes(include=[np.number])
X_test_cat = X_test.select_dtypes(include=[object])
X_test_num = X_test.select_dtypes(include=[np.number])

4. Encode the categorical variables X_train_cat and X_test_cat using the OneHotEncoder setup in the previous lab. Remember to use `.toarray()` after `.transform()` to endup with a numpy array. Next, cast the resulting numpy arrays into pandas DataFrames. Make sure that the column names of the new dataframes are correctly setup using `encoder.get_feature_names_out()` and the same indexes of X_train_cat and X_test_cat

In [8]:
from sklearn.preprocessing import OneHotEncoder

# Encode categorical variables
encoder = OneHotEncoder(handle_unknown='ignore').fit(X_train_cat)

X_train_cat_encoded = encoder.transform(X_train_cat).toarray()
X_test_cat_encoded = encoder.transform(X_test_cat).toarray()

# Create pandas dataframes for encoded categorical variables
X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded,
                                      columns=encoder.get_feature_names_out(X_train_cat.columns),
                                      index=X_train_cat.index)
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded,
                                     columns=encoder.get_feature_names_out(X_test_cat.columns),
                                     index=X_test_cat.index)

5. Use X_train_num to fit a power transformer. Transform BOTH X_train_num and X_test_num. Next, cast the resulting numpy arrays as pandas dataframes. Make sure to set the correct columns names and to use the same indexes of X_train_num and X_test_num. Name the final resulting dataframes as:
`X_train_num_transformed_df and X_test_num_transformed_df`

In [9]:
from sklearn.preprocessing import PowerTransformer

# Transform numerical variables
transformer = PowerTransformer().fit(X_train_num)

X_train_num_transformed = transformer.transform(X_train_num)
X_test_num_transformed = transformer.transform(X_test_num)

# Create pandas dataframes for transformed numerical variables
X_train_num_transformed_df = pd.DataFrame(X_train_num_transformed,
                                          columns=X_train_num.columns,
                                          index=X_train_num.index)
X_test_num_transformed_df = pd.DataFrame(X_test_num_transformed,
                                         columns=X_test_num.columns,
                                         index=X_test_num.index)

6. Concat X_train_num_transformed_df and X_train_cat_encoded_df into X_train_new and X_test_num_transformed_df and X_test_cat_encoded_df into X_test_new

In [10]:
X_train_new = pd.concat([X_train_num_transformed_df, X_train_cat_encoded_df], axis=1)
X_test_new = pd.concat([X_test_num_transformed_df, X_test_cat_encoded_df], axis=1)

7. Fit a MinMax scaler using X_train_new and transform X_train_new and X_test_new. Create new pandas dataframes from the resulting numpy arrays. Remember to set the correct columns names and indexes. Name the resulting dataframes as:
`X_train_new_scaled_df and X_test_new_scaled_df`

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_new_scaled = scaler.fit_transform(X_train_new)
X_test_new_scaled = scaler.transform(X_test_new)

# create pandas dataframes
X_train_new_scaled_df = pd.DataFrame(X_train_new_scaled, columns=X_train_new.columns, index=X_train_new.index)
X_test_new_scaled_df = pd.DataFrame(X_test_new_scaled, columns=X_test_new.columns, index=X_test_new.index)

8. Train a simple linear regression model using X_train_new_scaled_df, and get the predictions for the train and test sets

In [12]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train_new_scaled_df, y_train)

y_train_pred = lin_reg.predict(X_train_new_scaled_df)
y_test_pred = lin_reg.predict(X_test_new_scaled_df)

9. Create a function that given a model prediction and real values returns a pandas dataframe with the following table:

|Error_metric| Value |
|------------|-------|
| MAE        | value |
| MSE        | value |
| RMSE       | value |
| MAPE       | value |
| R2         | value |

In [13]:
import numpy as np

def evaluate_predictions(y_true, y_pred):
    mae = np.mean(np.abs(y_true - y_pred))
    mse = np.mean(np.square(y_true - y_pred))
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = 1 - np.sum(np.square(y_true - y_pred)) / np.sum(np.square(y_true - np.mean(y_true)))
    
    error_df = pd.DataFrame({
        'Error_metric': ['MAE', 'MSE', 'RMSE', 'MAPE', 'R2'],
        'Value': [mae, mse, rmse, mape, r2]
    })
    
    return error_df

10. Evaluate the linear model predictions using the previous function on the TRAIN and TEST sets

In [14]:
train_error_df = evaluate_predictions(y_train, y_train_pred)
test_error_df = evaluate_predictions(y_test, y_test_pred)

print('Train set errors:')
print(train_error_df)

print('\nTest set errors:')
print(test_error_df)

Train set errors:
  Error_metric         Value
0          MAE  1.189622e-12
1          MSE  2.431234e-24
2         RMSE  1.559242e-12
3         MAPE  1.370461e-12
4           R2  1.000000e+00

Test set errors:
  Error_metric         Value
0          MAE     98.245501
1          MSE  20961.284689
2         RMSE    144.780125
3         MAPE    110.345883
4           R2      0.759290


11. Now define a function that takes as an input: list of models, X_train and y_train to train several model (with default values) so we can train a lot of them without repeating code. The function must return the list of trained models.

In [15]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

def train_models(models, X_train, y_train):
    trained_models = []
    for model in models:
        trained_model = model.fit(X_train, y_train)
        trained_models.append(trained_model)
    return trained_models

models = [LinearRegression(), KNeighborsRegressor(), MLPRegressor()]

trained_models = train_models(models, X_train_new_scaled_df, y_train)

/Users/belma/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


12. Use the function to train the following models (with default settings):

*[LinearRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)

*[KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn-neighbors-kneighborsregressor)

*[MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor)

15. Evaluate the models with the function created earlier in the TRAIN and TEST sets. Which model performs best with the default options? 

In [16]:
def get_model_evaluation(predictions, y_true):
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    import numpy as np

    error_metrics = {}
    error_metrics['MAE'] = mean_absolute_error(y_true, predictions)
    error_metrics['MSE'] = mean_squared_error(y_true, predictions)
    error_metrics['RMSE'] = np.sqrt(mean_squared_error(y_true, predictions))
    error_metrics['MAPE'] = np.mean(np.abs((y_true - predictions) / y_true)) * 100
    error_metrics['R2'] = r2_score(y_true, predictions)

    error_metrics_df = pd.DataFrame(error_metrics.items(), columns=['Error_metric', 'Value'])
    return error_metrics_df

for model in trained_models:
    train_predictions = model.predict(X_train_new_scaled_df)
    test_predictions = model.predict(X_test_new_scaled_df)
    
    print(f'Model: {type(model).__name__}')
    print('Train set:')
    print(get_model_evaluation(train_predictions, y_train))
    print('Test set:')
    print(get_model_evaluation(test_predictions, y_test))
    print('-------------------------------------')
    
    

Model: LinearRegression
Train set:
  Error_metric         Value
0          MAE  1.189622e-12
1          MSE  2.431234e-24
2         RMSE  1.559242e-12
3         MAPE  1.370461e-12
4           R2  1.000000e+00
Test set:
  Error_metric         Value
0          MAE     98.245501
1          MSE  20961.284689
2         RMSE    144.780125
3         MAPE    110.345883
4           R2      0.759290
-------------------------------------
Model: KNeighborsRegressor
Train set:
  Error_metric         Value
0          MAE     87.500141
1          MSE  21491.505299
2         RMSE    146.599813
3         MAPE     97.680866
4           R2      0.741724
Test set:
  Error_metric         Value
0          MAE    114.065396
1          MSE  35345.378530
2         RMSE    188.003666
3         MAPE    108.365913
4           R2      0.594110
-------------------------------------
Model: MLPRegressor
Train set:
  Error_metric        Value
0          MAE    39.366306
1          MSE  6688.950516
2         RMSE    81

In [ ]:
model_list = [LinearRegression(), KNeighborsRegressor(), MLPRegressor()]

trained_models = train_models(model_list, X_train_new_scaled_df, y_train)

# Evaluate the models on train and test sets
train_evaluations = []
test_evaluations = []

for model in trained_models:
    # Make predictions on train and test sets
    y_train_pred = model.predict(X_train_new_scaled_df)
    y_test_pred = model.predict(X_test_new_scaled_df)
    
    # Evaluate model performance on train and test sets
    train_evaluations.append(evaluate_predictions(y_train, y_train_pred))
    test_evaluations.append(evaluate_predictions(y_test, y_test_pred))

In [ ]:
train_evaluations
#test_evaluations